In [1]:
import pandas as pd
import baseline_model as mb

In [2]:
train_data = pd.read_csv('/Users/finneyer/Documents/HSLU/Semester 3/DSPRO1/Projektarbeit/Data/0.2/London_UTD19_train_Sniper_0.csv')
test_data = pd.read_csv('/Users/finneyer/Documents/HSLU/Semester 3/DSPRO1/Projektarbeit/Data/0.2/London_UTD19_test_Sniper_0.csv')

In [ ]:
mb.train_random_baseline_models(train_data, '/Users/finneyer/Documents/HSLU/Semester 3/DSPRO1/Projektarbeit/models/baseline_2')

In [3]:
models_path = '/Users/finneyer/Documents/HSLU/Semester 3/DSPRO1/Projektarbeit/models/baseline_2'

In [ ]:
predictions = mb.get_random_baseline_prediction(models_path, 1)
predictions

In [4]:
evaluation = mb.evaluate_random_baseline_models(test_data, models_path)
evaluation

,Average MAE,Average MSE
0,5.666649,72.419063
